In this notebook different quantisation methods and distance metrics for Facial Recognition will be compared both on accuracy and execution time. 

The Quantisation methods include:
- Scalar Quantisation
- TensorFlow Quantisation

The distance metrics include:
- Cosine Similarity
- Euclidean Distance

Below are the necassary import to run the code.

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' # suppress tensorflow warnings https://stackoverflow.com/a/40871012
from deepface import DeepFace
import subprocess
import numpy as np
from decimal import Decimal # for proper rounding
import random
import time
import pandas as pd
from datetime import datetime
import struct
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import statistics
import accuracy as ac
import pickle
import quantisations as qt
import basics as bs


# CONSTANTS
EXECUTABLE_PATH = "ABY/build/bin"
INPUT_FILE_NAME = "input_vecs.txt"
EXECUTABLE_NAME_SCENARIO = 'cos_dist_copy'
CMD_SCENARIO = f"./{EXECUTABLE_NAME_SCENARIO} -r 1 -f {INPUT_FILE_NAME} & (./{EXECUTABLE_NAME_SCENARIO} -r 0 -f {INPUT_FILE_NAME} 2>&1 > /dev/null)"

# random number generator
rng = np.random.default_rng()

Here we test if quantisation works

In [2]:
embedding1 = bs.get_embedding("lfw/George_W_Bush/George_W_Bush_0001.jpg")
embedding2 = bs.get_embedding("lfw/George_W_Bush/George_W_Bush_0002.jpg")
bs.get_cos_dist_numpy(embedding1, embedding2)
# print(embedding1)
embedding1_quant=qt.scalar_quantisation_percentile(embedding1)
embedding2_quant=qt.quantize_tensor(embedding2)
print("size before scalar quantisation:" ,sys.getsizeof(embedding1), ", size after scalar quantisation:",sys.getsizeof(embedding1_quant)) 
print("size before tensor quantisation:", sys.getsizeof(embedding2), ", size after tensor quantisation:",sys.getsizeof(embedding2_quant)) 
print("the type of the elements in the scalar quantisation is:", type(embedding1_quant[0]), "in the non quantised embedding it was:",type(embedding1[0]))
print("the type of the elements in the tensor quantisation is:", type(embedding2_quant[0]), "in the non quantised embedding it was:",type(embedding2[0]))

size before scalar quantisation: 1080 , size after scalar quantisation: 240
size before tensor quantisation: 1080 , size after tensor quantisation: 240
the type of the elements in the scalar quantisation is: <class 'numpy.int8'> in the non quantised embedding it was: <class 'float'>
the type of the elements in the tensor quantisation is: <class 'numpy.int8'> in the non quantised embedding it was: <class 'float'>


Below are two functions to compare Facenet and Sface accuracy. One for Euclidean Distance and one for Cosine Similarity. The code to create a visual representation for this comparison is also included.

In [ ]:
### GENERATING THE GLOBAL PAIRS. 
### we only need to run this once and then we can have the file and use pairs as the list of embeddings 
### uncomment to use for first them, then use the next cell!

#
# Generate pairs globally
def generate_pairs(m):
    pairs = []
    for _ in range(m):
        print(_)
        n = random.choice([True, False])
        a, b, imga, imgb = bs.get_two_random_embeddings_facenet(same_person=n)
        pairs.append((a, b, imga, imgb,n))
    return pairs

m = 1000
pairs = generate_pairs(m)

# File path
file_path = 'embedingpairs.pkl'

# Delete the file if it exists
if os.path.exists(file_path):
    os.remove(file_path)

# Save pairs to a new file
with open(file_path, 'wb') as file:
    pickle.dump(pairs, file)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18


In [ ]:
with open('embedingpairs.pkl', 'rb') as file:
    pairs = pickle.load(file)

In [ ]:
def print_counters(counters):
    for quant_func, results in counters.items():
        print(f"\nQuantization Function: {quant_func}")
        for metric, count in results.items():
            print(f"{metric}: {count}")

def extract_results(counters):
    methods = []
    correct_sface_euc = []
    incorrect_sface_euc = []
    correct_sface_cos = []
    incorrect_sface_cos = []
    correct_facenet_euc = []
    incorrect_facenet_euc = []
    correct_facenet_cos = []
    incorrect_facenet_cos = []

    for method, counts in counters.items():
        methods.append(method)
        correct_sface_euc.append(counts['correct_tensor_sface'])
        incorrect_sface_euc.append(counts['wrong_tensor_sface'])
        correct_sface_cos.append(counts['correct_scalar_sface'])
        incorrect_sface_cos.append(counts['wrong_scalar_sface'])
        correct_facenet_euc.append(counts['correct_tensor_facenet'])
        incorrect_facenet_euc.append(counts['wrong_tensor_facenet'])
        correct_facenet_cos.append(counts['correct_scalar_facenet'])
        incorrect_facenet_cos.append(counts['wrong_scalar_facenet'])
    
    return (methods, correct_sface_euc, incorrect_sface_euc, correct_sface_cos, incorrect_sface_cos, correct_facenet_euc, incorrect_facenet_euc, correct_facenet_cos, incorrect_facenet_cos)

def visualize_results(methods, correct_sface_euc, incorrect_sface_euc, correct_sface_cos, incorrect_sface_cos, correct_facenet_euc, incorrect_facenet_euc, correct_facenet_cos, incorrect_facenet_cos):
    n_methods = len(methods)
    ind = np.arange(n_methods)
    width = 0.1       

    fig, ax = plt.subplots(figsize=(12, 8))

    bar1 = ax.bar(ind - width * 3.5, correct_sface_euc, width, label='Correct Euclidean (SFace)', color='skyblue')
    bar2 = ax.bar(ind - width * 2.5, incorrect_sface_euc, width, label='Incorrect Euclidean (SFace)', color='palevioletred')
    bar3 = ax.bar(ind - width * 1.5, correct_sface_cos, width, label='Correct Cosine (SFace)', color='orange')
    bar4 = ax.bar(ind - width * 0.5, incorrect_sface_cos, width, label='Incorrect Cosine (SFace)', color='purple')
    bar5 = ax.bar(ind + width * 0.5, correct_facenet_euc, width, label='Correct Euclidean (Facenet)', color='lightgreen')
    bar6 = ax.bar(ind + width * 1.5, incorrect_facenet_euc, width, label='Incorrect Euclidean (Facenet)', color='blueviolet')
    bar7 = ax.bar(ind + width * 2.5, correct_facenet_cos, width, label='Correct Cosine (Facenet)', color='lightcoral')
    bar8 = ax.bar(ind + width * 3.5, incorrect_facenet_cos, width, label='Incorrect Cosine (Facenet)', color='gold')

    for bars in [bar1, bar2, bar3, bar4, bar5, bar6, bar7, bar8]:
        for i, bar in enumerate(bars):
            ax.text(bar.get_x() + bar.get_width() / 2, bar.get_height() + 0.5, str(int(bar.get_height())), ha='center', va='bottom')

    ax.set_xlabel('Quantisation Methods')
    ax.set_ylabel('Values')
    ax.set_title('Accuracy Comparison of Different Quantisation Methods using SFace and Facenet (Euclidean Distance and Cosine Similarity)')
    ax.set_xticks(ind)
    ax.set_xticklabels(methods, rotation=45, ha='right')
    ax.legend(loc='upper right', bbox_to_anchor=(1.4, 1))

    plt.show()

# Running the comparison functions
counters_euc = ac.compare_accuracies_euc(pairs)
counters_cos = ac.compare_accuracies_cos(pairs)

# Print the counters for each function
print("Euclidean Distance Results:")
print_counters(counters_euc)

print("Cosine Similarity Results:")
print_counters(counters_cos)

# Extract results for visualization
(methods, correct_sface_euc, incorrect_sface_euc, correct_sface_cos, incorrect_sface_cos, correct_facenet_euc, incorrect_facenet_euc, correct_facenet_cos, incorrect_facenet_cos) = extract_results(counters_euc)

# Visualize the results
visualize_results(methods, correct_sface_euc, incorrect_sface_euc, correct_sface_cos, incorrect_sface_cos, correct_facenet_euc, incorrect_facenet_euc, correct_facenet_cos, incorrect_facenet_cos)


below will be the functions to compare the execution time of (Facenet, SFace) x (Euclidean, Cosine) X (no quantisation, Tensorflow, scalar)

In [ ]:
# Define your experiments
experiments = [
    {"n": 1000, "get_embedding_func": bs.get_embedding_facenet, "quantize_func": None, "distance_func": bs.get_cos_dist_numpy, "quantize": False},
    {"n": 1000, "get_embedding_func": bs.get_embedding_facenet, "quantize_func": None, "distance_func": bs.euclidean_distance, "quantize": False},
    {"n": 1000, "get_embedding_func": bs.get_embedding_facenet, "quantize_func": qt.scalar_quantisation_percentile, "distance_func": bs.get_cos_dist_numpy, "quantize": True},
    {"n": 1000, "get_embedding_func": bs.get_embedding_facenet, "quantize_func": qt.scalar_quantisation_percentile, "distance_func": bs.euclidean_distance, "quantize": True},
    {"n": 1000, "get_embedding_func": bs.get_embedding_facenet, "quantize_func": qt.quantize_tensor, "distance_func": bs.get_cos_dist_numpy, "quantize": True},
    {"n": 1000, "get_embedding_func": bs.get_embedding_facenet, "quantize_func": qt.quantize_tensor, "distance_func": bs.euclidean_distance, "quantize": True},
    {"n": 1000, "get_embedding_func": bs.get_embedding, "quantize_func": None, "distance_func": bs.get_cos_dist_numpy, "quantize": False},
    {"n": 1000, "get_embedding_func": bs.get_embedding, "quantize_func": None, "distance_func": bs.euclidean_distance, "quantize": False},
    {"n": 1000, "get_embedding_func": bs.get_embedding, "quantize_func": qt.scalar_quantisation_percentile, "distance_func": bs.get_cos_dist_numpy, "quantize": True},
    {"n": 1000, "get_embedding_func": bs.get_embedding, "quantize_func": qt.scalar_quantisation_percentile, "distance_func": bs.euclidean_distance, "quantize": True},
    {"n": 1000, "get_embedding_func": bs.get_embedding, "quantize_func": qt.quantize_tensor, "distance_func": bs.get_cos_dist_numpy, "quantize": True},
    {"n": 1000, "get_embedding_func": bs.get_embedding, "quantize_func": qt.quantize_tensor, "distance_func": bs.euclidean_distance, "quantize": True},
]

In [ ]:
def run_experiment(n, get_embedding_func, quantize_func, distance_func, quantize=False):
    execution_times = []

    for _ in range(n):
        a, b, imga, imgb,n = pairs[_]
        start_time = time.time()
        a = get_embedding_func(imga)
        b = get_embedding_func(imgb)
        
        if quantize:
            a = quantize_func(a)
            b = quantize_func(b)
        
        if distance_func == bs.get_cos_dist_numpy:
            a = a / np.linalg.norm(a)
            b = b / np.linalg.norm(b)
        
        distance_func(a, b)
        end_time = time.time()

        execution_time = end_time - start_time
        execution_times.append(execution_time)
    
    return execution_times

In [ ]:
# Run all experiments
results = {}
for i, experiment in enumerate(experiments):
    key = f"experiment_{i+1}"
    results[key] = run_experiment(**experiment)
    print(f"{key} completed")

In [ ]:
# Define a dictionary with experiment names for readability
experiment_names = {
    "experiment_1": "avg_execution_times_no_quantisation_facenet_cos",
    "experiment_2": "avg_execution_times_no_quantisation_facenet_ed",
    "experiment_3": "avg_execution_times_scalar_quantisation_facenet_cos",
    "experiment_4": "avg_execution_times_scalar_quantisation_facenet_ed",
    "experiment_5": "avg_execution_times_tensorflow_quantisation_facenet_cos",
    "experiment_6": "avg_execution_times_tensorflow_quantisation_facenet_ed",
    "experiment_7": "avg_execution_times_no_quantisation_sface_cos",
    "experiment_8": "avg_execution_times_no_quantisation_sface_ed",
    "experiment_9": "avg_execution_times_scalar_quantisation_sface_cos",
    "experiment_10": "avg_execution_times_scalar_quantisation_sface_ed",
    "experiment_11": "avg_execution_times_tensorflow_quantisation_sface_cos",
    "experiment_12": "avg_execution_times_tensorflow_quantisation_sface_ed",
}

In [ ]:
# Calculate and print the average execution times
for key, name in experiment_names.items():
    avg_time = statistics.mean(results[key])
    print(f"{name} = {avg_time}")

In [ ]:
# Define quantization functions
quantization_functions = {
    "scalar_quantisation_max": qt.scalar_quantisation_max,
    "scalar_quantisation_percentile": qt.scalar_quantisation_percentile,
    "scalar_quantisation_tensorrt": qt.scalar_quantisation_tensorrt
}

# Define your experiments
experiments = []

embedding_functions = [bs.get_embedding_facenet, bs.get_embedding]
distance_functions = [bs.get_cos_dist_numpy, bs.euclidean_distance]

# Add experiments for no quantization
for get_embedding_func in embedding_functions:
    for distance_func in distance_functions:
        experiments.append({
            "n": 1000,
            "get_embedding_func": get_embedding_func,
            "quantize_func": None,
            "distance_func": distance_func,
            "quantize": False
        })

# Add experiments for tensor quantization
for get_embedding_func in embedding_functions:
    for distance_func in distance_functions:
        experiments.append({
            "n": 1000,
            "get_embedding_func": get_embedding_func,
            "quantize_func": qt.quantize_tensor,
            "distance_func": distance_func,
            "quantize": True
        })

# Add experiments for each scalar quantization method
for get_embedding_func in embedding_functions:
    for distance_func in distance_functions:
        for name, quant_func in quantization_functions.items():
            experiments.append({
                "n": 1000,
                "get_embedding_func": get_embedding_func,
                "quantize_func": quant_func,
                "distance_func": distance_func,
                "quantize": True
            })

def run_experiment(n, get_embedding_func, quantize_func, distance_func, quantize=False):
    execution_times = []

    for _ in range(n):
        a, b, imga, imgb, n = pairs[_]
        start_time = time.time()
        a = get_embedding_func(imga)
        b = get_embedding_func(imgb)
        
        if quantize:
            a = quantize_func(a)
            b = quantize_func(b)
        
        if distance_func == bs.get_cos_dist_numpy:
            a = a / np.linalg.norm(a)
            b = b / np.linalg.norm(b)
        
        distance_func(a, b)
        end_time = time.time()

        execution_time = end_time - start_time
        execution_times.append(execution_time)
    
    return execution_times

# Run all experiments
results = {}
for i, experiment in enumerate(experiments):
    key = f"experiment_{i+1}"
    results[key] = run_experiment(**experiment)
    print(f"{key} completed")

# Define a dictionary with experiment names for readability
experiment_names = {
    "experiment_1": "avg_execution_times_no_quantisation_facenet_cos",
    "experiment_2": "avg_execution_times_no_quantisation_facenet_ed",
    "experiment_3": "avg_execution_times_no_quantisation_sface_cos",
    "experiment_4": "avg_execution_times_no_quantisation_sface_ed",
    "experiment_5": "avg_execution_times_tensor_quantisation_facenet_cos",
    "experiment_6": "avg_execution_times_tensor_quantisation_facenet_ed",
    "experiment_7": "avg_execution_times_tensor_quantisation_sface_cos",
    "experiment_8": "avg_execution_times_tensor_quantisation_sface_ed",
    "experiment_9": "avg_execution_times_scalar_quantisation_max_facenet_cos",
    "experiment_10": "avg_execution_times_scalar_quantisation_max_facenet_ed",
    "experiment_11": "avg_execution_times_scalar_quantisation_max_sface_cos",
    "experiment_12": "avg_execution_times_scalar_quantisation_max_sface_ed",
    "experiment_13": "avg_execution_times_scalar_quantisation_percentile_facenet_cos",
    "experiment_14": "avg_execution_times_scalar_quantisation_percentile_facenet_ed",
    "experiment_15": "avg_execution_times_scalar_quantisation_percentile_sface_cos",
    "experiment_16": "avg_execution_times_scalar_quantisation_percentile_sface_ed",
    "experiment_17": "avg_execution_times_scalar_quantisation_tensorrt_facenet_cos",
    "experiment_18": "avg_execution_times_scalar_quantisation_tensorrt_facenet_ed",
    "experiment_19": "avg_execution_times_scalar_quantisation_tensorrt_sface_cos",
    "experiment_20": "avg_execution_times_scalar_quantisation_tensorrt_sface_ed"
}

# Calculate and print the average execution times
for key, name in experiment_names.items():
    avg_time = statistics.mean(results[key])
    print(f"{name} = {avg_time}")
